In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv1D,Conv1DTranspose,Concatenate,Input,MaxPooling2D,MaxPooling1D,Conv2D,Conv2DTranspose
import numpy as np
import IPython.display as ipd
import glob
import librosa.display
import matplotlib.pyplot as plt
import time
from keras.models import load_model
import pandas as pd
# from tqdm.notebook import tqdm

In [ ]:
from google.colab import files
files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d srujannagamally/noise-cancellation

100% 2.00G/2.00G [01:46<00:00, 20.3MB/s]
100% 2.00G/2.00G [01:46<00:00, 20.1MB/s]


In [ ]:
!unzip noise-cancellation.zip

In [ ]:
def create_dataset(file_path,a=12000):
  start=time.time()
  set=glob.glob(file_path)
  data=np.array(librosa.load(set[0])[0])
  for i in range(1,len(set)):
    add=np.array(librosa.load(set[i])[0])
    data=tf.concat([data,add],0)
    print('1/2 ',i*100/len(set),'%')
    ipd.clear_output(wait=True)
  dataset=data[:a]
  dataset=np.reshape(dataset,(1,a,1))
  for i in range(1,int(data.shape[0]/a)):
    dataset=tf.concat([dataset,np.reshape(data[i*a:a*(i+1)],(1,a,1))],axis=0)
    print('2/2 ',(1+i)*a*100/len(data))
    ipd.clear_output(wait=True)
  end=time.time()
  time_taken=end-start
  print("time taken:",time_taken)
  return dataset

In [ ]:
def create_stft_dataset(x_train,a=12000):
  x_train_stft=librosa.stft(np.reshape((array),(x_train.shape[0],a,)),n_fft=256,hop_length=None)
  x_train_stft_db=librosa.amplitude_to_db(np.abs(x_train_stft))
  return x_train_stft_db

In [ ]:
def mel_dataset(x_test,a=12000):
  start=time.time()
  array=np.array(x_test[0])
  array=np.reshape(array,(a))
  x_test_mel=librosa.feature.melspectrogram(y=np.array(array),sr=22050,n_mels=256)
  b=x_test_mel.shape[1]
  x_test_mel=np.reshape(x_test_mel,(1,256,b))
  for i in range(1,x_test.shape[0]):
    array=np.array(x_test[i])
    array=np.reshape(array,(a))
    S = librosa.feature.melspectrogram(y=np.array(array),sr=22050,n_mels=256)
    S_db_mel = librosa.amplitude_to_db(S)
    x_test_mel=tf.concat([x_test_mel,np.reshape(S_db_mel,(1,256,b))],axis=0)
    print(i*100/x_test.shape[0],'%')
    ipd.clear_output(wait=True)
  end=time.time()
  time_taken=end-start
  print("time taken:",time_taken)
  return x_test_mel

In [ ]:
def save_dataset(file_name,dataset):
  np.save(f'/content/drive/MyDrive/Colab Notebooks/noise download/{file_name}.npy',np.array(dataset))

In [ ]:
# x_test=create_dataset('/content/noisy_testset_wav/noisy_testset_wav/*.wav')

In [ ]:
# save_dataset('x_test',x_test)

In [ ]:
# x_train_mel1=mel_dataset(x_train)

In [ ]:
# save_dataset("y_train_stft_db.npy",create_stft_dataset(y_train))

In [ ]:
# device=tf.test.gpu_device_name()
# device

'/device:GPU:0'

In [ ]:
x_train=np.load('/content/drive/MyDrive/Colab Notebooks/noise download/x_train.npy')
y_train=np.load('/content/drive/MyDrive/Colab Notebooks/noise download/y_train.npy')
x_train.shape

(62123, 12000, 1)

In [ ]:
a=12000

In [ ]:
array=np.reshape(x_train[0],(a))
plt.figure(figsize=(12, 4))
librosa.display.waveshow(array)
plt.show()

In [ ]:
array=np.reshape(y_train[0],(a))
plt.figure(figsize=(12, 4))
librosa.display.waveshow(array)
plt.show()

In [ ]:
def calculate_psnr(original_signal, reconstructed_signal):
    mse = np.mean((original_signal - reconstructed_signal) ** 2)
    max_value = np.max(original_signal)
    psnr = 10 * np.log10((max_value ** 2) / mse)
    return psnr

In [ ]:
def calculate_mse(sound1,sound2):
    squared_diff = (sound1 - sound2) ** 2
    mse = np.mean(squared_diff)
    return mse

In [ ]:
def calculate_snr(signal, noise):
    power_signal = np.mean(np.square(signal))
    power_noise = np.mean(np.square(noise))

    snr = 10 * np.log10(power_signal / power_noise)
    return snr

In [ ]:
# model1
inp = Input(shape=(12000,1))
c1 = Conv1D(2,32,2,'same',activation='relu')(inp)
c2 = Conv1D(4,32,2,'same',activation='relu')(c1)
c3 = Conv1D(8,32,2,'same',activation='relu')(c2)
c4 = Conv1D(16,32,2,'same',activation='relu')(c3)
c5 = Conv1D(32,32,2,'same',activation='relu')(c4)

dc1 = Conv1DTranspose(32,32,1,padding='same')(c5)
conc = Concatenate()([c5,dc1])
dc2 = Conv1DTranspose(16,32,2,padding='same')(conc)
conc = Concatenate()([c4,dc2])
dc3 = Conv1DTranspose(8,32,2,padding='same')(conc)
conc = Concatenate()([c3,dc3])
dc4 = Conv1DTranspose(4,32,2,padding='same')(conc)
conc = Concatenate()([c2,dc4])
dc5 = Conv1DTranspose(2,32,2,padding='same')(conc)
conc = Concatenate()([c1,dc5])
dc6 = Conv1DTranspose(1,32,2,padding='same')(conc)
conc = Concatenate()([inp,dc6])
dc7 = Conv1DTranspose(1,32,1,padding='same',activation='linear')(conc)
model1 = tf.keras.models.Model(inp,dc7)

![](https://drive.google.com/uc?export=view&id=1H9tND7PiyCYzVTr33GPxk41yrqFD9KXQ)

![](https://drive.google.com/uc?export=view&id=1mKB4wVES-zBeYegXE6rWTZIJ7P-wN9YR)

In [ ]:
model1.summary()

In [ ]:
# tf.keras.utils.plot_model(model1,show_shapes=True,show_layer_names=False)

In [ ]:
LOSS=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanSquaredError()]
# METRIC=[calculate_psnr]

In [ ]:
model1.compile(optimizer=tf.keras.optimizers.Adam(0.002),loss=LOSS)
history1 = model1.fit(x_train,y_train,epochs=20,validation_split=0.1)

In [ ]:
model1.save('/content/drive/MyDrive/Colab Notebooks/noise download/model1.h5')

In [ ]:
x_test=np.load('/content/drive/MyDrive/Colab Notebooks/noise download/x_test.npy')
y_test=np.load('/content/drive/MyDrive/Colab Notebooks/noise download/y_test.npy')

In [ ]:
predict1=model1.predict(x_test)

In [ ]:
np.save('/content/drive/MyDrive/Colab Notebooks/noise download/predict1.npy',predict1)

In [ ]:
model1.evaluate(x_test,y_test)

In [ ]:
import pickle
pickle.dump(model1, open('/content/drive/MyDrive/Colab Notebooks/noise download/model1.pkl','wb'))

In [ ]:
# model2
inp = Input(shape=(256,24,1))
c1 = Conv2D(2,(3,3),1,'same',activation='relu')(inp)
c2 = Conv2D(4,(3,3),1,'same',activation='relu')(c1)
c3 = Conv2D(8,(3,3),1,'same',activation='relu')(c2)
c4 = Conv2D(16,(3,3),1,'same',activation='relu')(c3)
c5 = Conv2D(32,(3,3),1,'same',activation='relu')(c4)

dc1 = Conv2DTranspose(32,(3,3),1,padding='same')(c5)
dc2 = Conv2DTranspose(16,(3,3),1,padding='same')(dc1)
dc3 = Conv2DTranspose(8,(3,3),1,padding='same')(dc2)
dc4 = Conv2DTranspose(4,(3,3),1,padding='same')(dc3)
dc5 = Conv2DTranspose(2,(3,3),1,padding='same')(dc4)
dc6 = Conv2DTranspose(1,(3,3),1,padding='same')(dc5)
dc7 = Conv2DTranspose(1,(3,3),1,padding='same',activation='linear')(dc6)
model2 = tf.keras.models.Model(inp,dc7)
model2.summary()

![](https://drive.google.com/uc?export=view&id=1y6Cky1-fo82vEFsu3Pa_kmSitM_IOwby)

![](https://drive.google.com/uc?export=view&id=1XjymCWPWtIu2IguUSYg4buNskRUb3V54)

NOISE

![](https://drive.google.com/uc?export=view&id=1SHkMPHf-QNF8QEnxS1t5MuXlM6Q-Py9o)

PREDICTION

![](https://drive.google.com/uc?export=view&id=18aBMJ_y-eY5TJeTXf6Mx5NSbPxlN2GZL)

METRICS

![](https://drive.google.com/uc?export=view&id=1BWQ6o-TSVomzSp9cIFLdgd7JybsKxNjY)

In [ ]:
# model3
inp = Input(shape=(a,1))
c1 = Conv1D(2,32,1,'same',activation='relu')(inp)
c2 = Conv1D(4,32,1,'same',activation='relu')(c1)
c3 = Conv1D(8,32,1,'same',activation='relu')(c2)
c4 = Conv1D(16,32,1,'same',activation='relu')(c3)
c5 = Conv1D(32,32,1,'same',activation='relu')(c4)

dc1 = Conv1DTranspose(32,32,1,padding='same')(c5)
dc2 = Conv1DTranspose(16,32,1,padding='same')(dc1)
dc3 = Conv1DTranspose(8,32,1,padding='same')(dc2)
dc4 = Conv1DTranspose(4,32,1,padding='same')(dc3)
dc5 = Conv1DTranspose(2,32,1,padding='same')(dc4)
dc6 = Conv1DTranspose(1,32,1,padding='same')(dc5)
dc7 = Conv1DTranspose(1,32,1,padding='same',activation='linear')(dc6)
model2 = tf.keras.models.Model(inp,dc7)

![](https://drive.google.com/uc?export=view&id=1Ww22U_28FXLVuzfvjbS64zh1kH3GNXfJ)

NOISE

![](https://drive.google.com/uc?export=view&id=1UMcVqbtIh-cMxv8ze61qaCe3XuikRuos)

PREDICTION

![](https://drive.google.com/uc?export=view&id=1VDcHfCkYKAMPhAGpizZO3_kHWyTwewq2)

CLEAN

![](https://drive.google.com/uc?export=view&id=1HFaPXq94zo2MztQYrSGR3R8HdnhvfZ5A)

METRICS

![](https://drive.google.com/uc?export=view&id=1yHOmNZI3kljqSz5f9xv5e82tgiV9m-tY)

NOISE

![](https://drive.google.com/uc?export=view&id=1-pBNsqacXKHOJ1aLvQQswkWEUQujnsvM)

PREDICTION

![](https://drive.google.com/uc?export=view&id=18R_YEBrBV0N4J8sx9hzTxlVnVuRiU1Sn)

CLEAN

![](https://drive.google.com/uc?export=view&id=1qXMoOU5uvU-buPhF9AX2oqefW1YXnrCJ)

METRICS

![](https://drive.google.com/uc?export=view&id=1GeNJYcF7nyyTzSQc8nm185V4b5qBGrUi)

NOISE

![](https://drive.google.com/uc?export=view&id=1Y8GgmwgEsxrtrsO7k-Hvh0QF8JIwTKOl)

PREDICTION

![](https://drive.google.com/uc?export=view&id=1jvm_uKzBEqM1iv9XRo43GNA6RDjV5wHR)

METRICS

![](https://drive.google.com/uc?export=view&id=1OyL17H5tngopbWsVktUf4TTAAjJAdBDa)

In [ ]:
# model4
model4 = tf.keras.Sequential()
model4.add(layers.SimpleRNN(173, return_sequences=True,input_shape=(256,173)))
model4.add(layers.SimpleRNN(300, return_sequences=True))
model4.add(layers.SimpleRNN(173, return_sequences=True))
model4.add(layers.Dense(173))
model4.summary()

NOISE

![](https://drive.google.com/uc?export=view&id=1tkkPKIEEvtiJWzDt4DzreXdLym3KBmde)

PREDICTION

![](https://drive.google.com/uc?export=view&id=14sTutqstiKMVXqL-8TI8u8BnSwKNIoY1)

METRICS

![](https://drive.google.com/uc?export=view&id=13cvrY-jAnwVWb9UPd1gVxM9_BC9WUvSU)

In [ ]:
# model5
model = tf.keras.Sequential()
model.add(layers.Dense(12000,input_dim=12000))
model.summary()

![](https://drive.google.com/uc?export=view&id=1nF6lcotrelbZCT_N96Q1Ny-qvB_MhVgV)

![](https://drive.google.com/uc?export=view&id=1A0os8rfyoY5ru9Dvk2LiC-C8A-JdbSVi)

In [ ]:
# model6
inp = Input(shape=(88200,1))
c1 = Conv1D(2,32,1,'same',activation='relu')(inp)
c2 = Conv1D(4,32,1,'same',activation='relu')(c1)
c3 = Conv1D(8,32,1,'same',activation='relu')(c2)
c4 = Conv1D(16,32,1,'same',activation='relu')(c3)
c5 = Conv1D(32,32,1,'same',activation='relu')(c4)

dc1 = Conv1DTranspose(32,32,1,padding='same')(c5)
dc2 = Conv1DTranspose(16,32,1,padding='same')(dc1)
dc3 = Conv1DTranspose(8,32,1,padding='same')(dc2)
dc4 = Conv1DTranspose(4,32,1,padding='same')(dc3)
dc5 = Conv1DTranspose(2,32,1,padding='same')(dc4)
dc6 = Conv1DTranspose(1,32,1,padding='same')(dc5)
dc7 = Conv1DTranspose(1,32,1,padding='same',activation='linear')(dc6)
model2 = tf.keras.models.Model(inp,dc7)

In [ ]:
# model7
inp = Input(shape=(256,173,1))
c1 = Conv2D(2,(3,3),1,'same',activation='relu')(inp)
c2 = Conv2D(4,(3,3),1,'same',activation='relu')(c1)
c3 = Conv2D(8,(3,3),1,'same',activation='relu')(c2)
c4 = Conv2D(16,(3,3),1,'same',activation='relu')(c3)
c5 = Conv2D(32,(3,3),1,'same',activation='relu')(c4)

dc1 = Conv2DTranspose(32,(3,3),1,padding='same')(c5)
dc2 = Conv2DTranspose(16,(3,3),1,padding='same')(dc1)
dc3 = Conv2DTranspose(8,(3,3),1,padding='same')(dc2)
dc4 = Conv2DTranspose(4,(3,3),1,padding='same')(dc3)
dc5 = Conv2DTranspose(2,(3,3),1,padding='same')(dc4)
dc6 = Conv2DTranspose(1,(3,3),1,padding='same')(dc5)
dc7 = Conv2DTranspose(1,(3,3),1,padding='same',activation='linear')(dc6)
model = tf.keras.models.Model(inp,dc7)
model.summary()

![](https://drive.google.com/uc?export=view&id=1x97PCSkT-oAMyy9G_jfqENfzKPsMLXDC)

NOISE

![](https://drive.google.com/uc?export=view&id=1SpS7liqkO4hh40LXtlPQa56VK4pXFKX-)

PREDICTION

![](https://drive.google.com/uc?export=view&id=1dJII_JZtQXlqRf8knxp0Cal_YY545DUW)

METRICS

![](https://drive.google.com/uc?export=view&id=1aeaeQOR_kyg8SFYp82Isrkyi415lbx7r)

In [ ]:
# model8
model8 = tf.keras.Sequential()
model8.add(layers.SimpleRNN(188, return_sequences=True,input_shape=(129,188)))
model8.add(layers.SimpleRNN(188, return_sequences=True))
model8.summary()

In [ ]:
# model9
model9 = tf.keras.Sequential()
model9.add(layers.SimpleRNN(188, return_sequences=True,input_shape=(129,188)))
model9.add(layers.SimpleRNN(300, return_sequences=True))
model9.add(layers.SimpleRNN(188, return_sequences=True))
model9.add(layers.Dense(188))
model9.summary()

![](https://drive.google.com/uc?export=view&id=1WXd1zlmRBnyC2-MvaipjjWJvPNlJmEgt)

NOISE

![](https://drive.google.com/uc?export=view&id=1Nbt915d-2YQxdtZRylufc0rGlWbNiyib)

METRICS

![](https://drive.google.com/uc?export=view&id=1wjMsoeJh2CSW-sL6kiAGNIivbkWY5y9J)

In [ ]:
# model10
model10 = tf.keras.Sequential()
model10.add(layers.LSTM(188, return_sequences=True,input_shape=(129,188)))
model10.add(layers.LSTM(300, return_sequences=True))
model10.add(layers.LSTM(188, return_sequences=True))
model10.add(layers.Dense(188))
model10.summary()

![](https://drive.google.com/uc?export=view&id=1203doIN4NmDzEjeJ_doRgjH-K6VdnKzc)

NOISE

![](https://drive.google.com/uc?export=view&id=1qZY60WuepAiMAza_387OkvoJHsFI-G0Y)

PREDICTION

![](https://drive.google.com/uc?export=view&id=1VMgXy3X3iteQix7N_uXfVSD1XFFKRygx)

METRICS

![](https://drive.google.com/uc?export=view&id=1M3UFI2n0X0T0dG4k7LNugsI7y5kftpV4)

In [ ]:
# model11
model11 = tf.keras.Sequential()
model11.add(layers.GRU(188, return_sequences=True,input_shape=(129,188)))
model11.add(layers.GRU(300, return_sequences=True))
model11.add(layers.GRU(188, return_sequences=True))
model11.add(layers.Dense(188))
model11.summary()

![](https://drive.google.com/uc?export=view&id=1YwX2V28BUIja0CYjBUih7nTUyWywxpi0)

![](https://drive.google.com/uc?export=view&id=1WkKLiZUVFv_9nyKHMwVfMl6ecu86vboY)

NOISE

![](https://drive.google.com/uc?export=view&id=1qXVULl_iQ3GRcakvRhuJ-NCpuU8A55B4)

PREDICTION

![](https://drive.google.com/uc?export=view&id=18NxDjSvZgc3xrtE9d0q-XqLDHaJNA6Ah)

METRICS

![](https://drive.google.com/uc?export=view&id=174qj95T4o7yLvAa2rNth91lXIqjCMnqU)

In [ ]:
# model12(model)
model = tf.keras.Sequential()
# model.add(layers.Embedding(input_dim=1000, output_dim=64))
# model.add(layers.GRU(512, return_sequences=True,input_dim=12000))
model.add(layers.SimpleRNN(512,input_dim=12000))
model.add(layers.Dense(12000))

model.summary()

![](https://drive.google.com/uc?export=view&id=1lUPr1EFPaTrtNSH5qZKxf2LxYMBAmJfA)

NOISE

![](https://drive.google.com/uc?export=view&id=1EhzYeWVaGd5O_7SMNDrFi3vuwQsiuaXY)

PREDICTION

![](https://drive.google.com/uc?export=view&id=1ESIRadSLCnYbaxdrL8XuO9NC0ddKXSjJ)

CLEAN

![](https://drive.google.com/uc?export=view&id=1Q2yD_Vde3pFvOgymkVmgWWROWH6Ubln2)

METRICS

![](https://drive.google.com/uc?export=view&id=1xKHGIMYYABlZui1IxFtDhq24KZAXhBKh)

NOISE

![](https://drive.google.com/uc?export=view&id=1y5gAiI-JTr0aHkPE-fMh2NhWYvaV5bq0)

PREDICTION

![](https://drive.google.com/uc?export=view&id=1d2c1J7C4fCwQB266iWh-pb9TrFc63YZ1)

CLEAN

![](https://drive.google.com/uc?export=view&id=1ykvxqHVf7rCn0vTHDGbXXHvrYYCYE27_)

METRICS

![](https://drive.google.com/uc?export=view&id=1ulHoVxrWFykI6GG175e1Y14h_qBox03Q)

In [ ]:
import matplotlib.pyplot as plt

training_loss = history1.history['loss']
validation_loss = history1.history['val_loss']

epochs = range(1, len(training_loss) + 1)
plt.figure(figsize=(10, 4))

plt.plot(epochs, training_loss, 'b-', label='Training Loss')
plt.plot(epochs, validation_loss, 'r-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
def create_predict_dataset(data,a=12000):
  dataset=data[:a]
  dataset=np.reshape(dataset,(1,a,1))
  for i in range(1,int(data.shape[0]/a)):
    dataset=tf.concat([dataset,np.reshape(data[i*a:a*(i+1)],(1,a,1))],axis=0)
  dataset=tf.concat([dataset,np.reshape(data[-a:],(1,a,1))],axis=0)
  diff=len(data)%a
  return dataset,diff

In [ ]:
def predict_dataset(dataset,diff,a=12000):
  predict=model2.predict(dataset)
  predict=predict.flatten()
  predict=tf.concat([predict[:-a],predict[-diff:]],axis=0)
  return predict

In [ ]:
def predict_from_mel(dataset,diff,a=12000):
  predict=model.predict(dataset)
  predict=librosa.db_to_amplitude(predict)
  predict=librosa.feature.inverse.mel_to_istft(predict)
  predict=librosa.istft(predict)
  predict=tf.concat([predict[:-a],predict[-diff:]],axis=0)
  return predict

In [ ]:
def predict_from_stft(dataset,diff,a=12000):
  predict=model.predict(dataset)
  predict=librosa.db_to_amplitude(predict)
  predict=librosa.istft(predict)
  predict=tf.concat([predict[:-a],predict[-diff:]],axis=0)
  return predict

In [ ]:
data=librosa.load('/content/drive/MyDrive/Colab Notebooks/noise download/noise1.wav')[0]
dataset,diff=create_predict_dataset(data)
p=predict(dataset,diff)